<a href="https://colab.research.google.com/github/Gangadharan-S/Multiclass-Fish-Image-Classification/blob/main/multifishstreamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -q streamlit pyngrok tensorflow pillow pandas

In [25]:
%%writefile app.py
import streamlit as st
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import pandas as pd

@st.cache_resource
def load_best_model():
    model_path = "/content/drive/MyDrive/Project/best_fish_model.h5"
    return load_model(model_path)

best_model = load_best_model()

class_names = [
    "Fish", "Fish Bass", "Black Sea Spart", "Gilt Heard Bream", "Horse Mackerel",
    "Red Mullet", "Red Sea Bream", "Sea Bass", "Shrimp", "Striped Red Mullet", "Trout"
]

def preprocess_image(image):
    image = image.convert("RGB")
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

st.title("🐠 Fish Classification App")
st.write("Upload images of fish, and the model will predict their types!")

st.sidebar.header("⚙ Settings")
confidence_threshold = st.sidebar.slider("Confidence Threshold (%)", 0, 100, 50, 5)

uploaded_files = st.file_uploader("📷 Choose images...", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

if uploaded_files:
    for uploaded_file in uploaded_files:
        st.divider()
        st.subheader(f"📷 Image: {uploaded_file.name}")
        image = Image.open(uploaded_file)
        st.image(image, caption="📸 Uploaded Image", use_container_width=True)
        processed_image = preprocess_image(image)

        with st.spinner("🔍 Classifying..."):
            prediction = best_model.predict(processed_image)
            predicted_class = class_names[np.argmax(prediction)]
            confidence = np.max(prediction) * 100

        if confidence >= confidence_threshold:
            st.success(f"🎯 *Prediction: {predicted_class}*")
            st.write(f"🔵 *Confidence:* {confidence:.2f}%")
        else:
            st.warning("⚠ Prediction confidence is too low. Try uploading a clearer image.")

        st.subheader("📊 Class Probabilities")
        prob_dict = {class_names[i]: f"{pred*100:.2f}%" for i, pred in enumerate(prediction[0])}
        st.json(prob_dict)

        prob_df = pd.DataFrame({"Class": class_names, "Confidence (%)": prediction[0] * 100})
        prob_df = prob_df.sort_values(by="Confidence (%)", ascending=False)
        st.bar_chart(prob_df.set_index("Class"))


Overwriting app.py


In [27]:
from pyngrok import conf, ngrok

conf.get_default().auth_token = "2zSwv7MhadWyFUgXnQINzLgNLQk_6ALkEGghZTMkRNFwyDKNW"

# Step 2: Kill previous tunnels
ngrok.kill()

public_url = ngrok.connect("http://localhost:8501", proto="http")
print(f"🌍 Your Streamlit app is live at: {public_url}")


# Step 4: Start the Streamlit app
!streamlit run app.py --server.port 8501 --server.enableCORS false


🌍 Your Streamlit app is live at: NgrokTunnel: "https://576d-34-91-97-84.ngrok-free.app" -> "http://localhost:8501"
2025-07-05 18:09:50.307 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.97.84:8501

2025-07-05 18:11:51.115098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751739111.143215   17260 cuda_dnn.cc:8310] Unable to reg